# Load raw data into Neptune

## Import Packages

In [ ]:
import subprocess
import pandas as pd
import boto3
import psycopg2
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
import pandas as pd
import psycopg2
import os

import igraph as ig
import json
from networkx.readwrite import json_graph

from __future__  import print_function  # Python 2/3 compatibility

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Cardinality

import boto3
from boto3 import client
from io import StringIO

## Pull in and merge movies data

In [2]:
#GENERATE TOKENS FOR RDS ACCESS

rds_token_cmd = f'aws rds generate-db-auth-token --hostname "mgt-prd-infr-rds-uscis-cluster-0.cargfitmjuj5.us-east-1.rds.amazonaws.com" --port 5432 --region us-east-1 --username "rdspgadm"'
rds_token = str(subprocess.check_output(rds_token_cmd, shell=True, text=True)).strip()

In [ ]:
#CONNTECT TO RDS SERVER

conn = psycopg2.connect(
    host="mgt-prd-infr-rds-uscis-cluster-0.cargfitmjuj5.us-east-1.rds.amazonaws.com",
    database="postgres",
    user="rdspgadm",
    password=rds_token)

In [3]:
#TESTING CONNECTION: OUTPUT ALL AVAILABLE TABLES

cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

('movietweets_movies',)
('imdb_name_basics',)
('imdb_title_basic',)
('imdb_title_crew',)
('imdb_title_episode',)
('imdb_title_ratings',)
('imdb_title_akas',)
('wiki_movies_test',)
('wiki_entities',)
('wiki_topics',)
('wiki_sentiments',)
('wiki_movies_5000',)
('imdb_dataset',)
('wiki_movies_10000',)
('wiki_movies_15000',)
('wiki_movies_19300',)
('merged_movie_data',)
('dossier_table',)
('cncc_0621_hashed_2012',)
('cncc_0621_hashed_2013',)
('cncc_0621_hashed_2014',)
('cncc_0621_hashed_2015',)
('cncc_0621_hashed_2016',)
('cncc_0621_hashed_2017',)
('cncc_0621_hashed_2018',)
('cncc_0621_hashed_2019',)
('wiki_movies_all',)
('merged_movie_data2',)
('cncc_0621_hashed_2020',)
('cncc_0621_hashed_2021',)
('cnhu_0621_hashed_2019',)
('cnhu_0621_hashed_2020',)
('cnhu_0621_hashed_2021',)
('wiki_movies',)


#### Bring in Wiki Data

In [4]:
df = pd.read_sql("""SELECT * FROM public.dossier_table""", con=conn)

In [116]:
pd.set_option('display.max_columns', None)
df.head()

,title,tconst,year,averagerating,numvotes,director,writers,runtimeminutes,startyear,originaltitle,primarytitle,genres,endyear,titletype,isadult,starring,release_date,producer,categories,summary,plot,polarity,subjectivity,topic_id,topic_prob,topic,characters
0,The Other Side of the Wind (2018),tt0069049,2018,6.7,7153,['Orson Welles'],"nm0000080,nm0462648",122.0,2018,The Other Side of the Wind,The Other Side of the Wind,Drama,None,movie,0,"['John Huston', 'Peter Bogdanovich', 'Susan St...",None,None,"['2010s satirical films', '2018 films', 'All a...",The Other Side of the Wind is a 2018 film dire...,The film opens by describing the final day of ...,NaN,NaN,NaN,NaN,None,None
1,Pál Adrienn (2010),tt0146592,2010,6.7,510,['Ágnes Kocsis'],"nm1030585,nm1921430",136.0,2010,Pál Adrienn,Pál Adrienn,Drama,None,movie,0,['Éva Gábor'],None,None,"['2010 drama films', '2010 films', '2010s dram...",Pál Adrienn is a 2010 Hungarian drama film dir...,None,NaN,NaN,NaN,NaN,None,None
2,Foodfight! (2012),tt0249516,2012,1.8,10750,['Lawrence Kasanoff'],"nm0440415,nm0923312,nm0295165,nm0841854,nm0220297",91.0,2012,Foodfight!,Foodfight!,"Action,Adventure,Animation",None,movie,0,"['Charlie Sheen', 'Eva Longoria', 'Hilary Duff...",None,\nGeorge Johnsen\nLawrence Kasanoff\nJoshua We...,"['2010s American animated films', '2010s adven...",Foodfight! is a 2012 American computer animate...,When night falls at the supermarket Marketropo...,NaN,NaN,NaN,NaN,None,None
3,Chinango (2009),tt0312958,2009,5.4,72,Peter Van Lengen,"nm1691085,nm1138779",90.0,2009,Chinango,Chinango,"Action,Comedy",None,movie,0,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None
4,Wazir (2016),tt0315642,2016,7.1,18786,Bejoy Nambiar,"nm0006765,nm0430785,nm2349060,nm5241801,nm1249...",103.0,2016,Wazir,Wazir,"Action,Crime,Drama",None,movie,0,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None


## Create edge and node list

#### Pull out entities

In [6]:
def splitter(s):
    spl = s.split(",")
    return spl

Director

In [246]:
df_dir = df[['title', 'director']]
df_dir['director'] = df_dir['director'].str.replace(r"[\"\'\\[\]]", '')
df_dir = df_dir[df_dir['director'].notnull()]
df_dir = df_dir.reset_index(drop=True)
df_dir.head(10)

<ipython-input-246-6a05704d9cd7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['director'] = df_dir['director'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-246-6a05704d9cd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir['director'] = df_dir['director'].str.replace(r"[\"\'\\[\]]", '')


,title,director
0,The Other Side of the Wind (2018),Orson Welles
1,Pál Adrienn (2010),Ágnes Kocsis
2,Foodfight! (2012),Lawrence Kasanoff
3,Chinango (2009),Peter Van Lengen
4,Wazir (2016),Bejoy Nambiar
5,The Wicker Tree (2011),Robin Hardy
6,Coraline (2009),Henry Selick
7,On the Road (2012),Walter Salles
8,The Future (2011),Miranda July
9,The Watch (2012),Akiva Schaffer


In [247]:
df_new = pd.DataFrame(df_dir["director"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [248]:
df_melt

,title,director,0,1,2,3,4,5,6,7,8
0,The Other Side of the Wind (2018),Orson Welles,Orson Welles,None,None,None,None,None,None,None,None
1,Pál Adrienn (2010),Ágnes Kocsis,Ágnes Kocsis,None,None,None,None,None,None,None,None
2,Foodfight! (2012),Lawrence Kasanoff,Lawrence Kasanoff,None,None,None,None,None,None,None,None
3,Chinango (2009),Peter Van Lengen,Peter Van Lengen,None,None,None,None,None,None,None,None
4,Wazir (2016),Bejoy Nambiar,Bejoy Nambiar,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
16816,Pegasus (2019),Han Han,Han Han,None,None,None,None,None,None,None,None
16817,Friend Zone (2019),Chayanop Boonprakob,Chayanop Boonprakob,None,None,None,None,None,None,None,None
16818,Castle of Dreams (2019),Wilfred Noy,Wilfred Noy,None,None,None,None,None,None,None,None
16819,Beyond the Visible - Hilma af Klint (2019),Halina Dyrschka,Halina Dyrschka,None,None,None,None,None,None,None,None


In [249]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [250]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
directors = df_melt_long
directors["edge_type"] = "Directed"
directors["from_type"] = "Movie"
directors["to_type"] = "Director"

In [251]:
directors.head()

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),Orson Welles,Directed,Movie,Director
1,Pál Adrienn (2010),Ágnes Kocsis,Directed,Movie,Director
2,Foodfight! (2012),Lawrence Kasanoff,Directed,Movie,Director
3,Chinango (2009),Peter Van Lengen,Directed,Movie,Director
4,Wazir (2016),Bejoy Nambiar,Directed,Movie,Director


Starring

In [252]:
df_dir = df[['title', 'starring']]
df_dir['starring'] = df_dir['starring'].str.replace(r"[\"\'\\[\]]", '')
df_dir = df_dir[df_dir['starring'].notnull()]
df_dir = df_dir.reset_index(drop=True)
df_dir.head(10)

<ipython-input-252-5065d894368f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['starring'] = df_dir['starring'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-252-5065d894368f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir['starring'] = df_dir['starring'].str.replace(r"[\"\'\\[\]]", '')


,title,starring
0,The Other Side of the Wind (2018),"John Huston, Peter Bogdanovich, Susan Strasber..."
1,Pál Adrienn (2010),Éva Gábor
2,Foodfight! (2012),"Charlie Sheen, Eva Longoria, Hilary Duff, Wayn..."
3,The Wicker Tree (2011),"Christopher Lee, Graham McTavish, Jacqueline L..."
4,Blood and Bone (2009),"Julian Sands, Eamonn Walker, Dante Basco, Nona..."
5,Inglourious Basterds (2009),"Brad Pitt, Christoph Waltz, Michael Fassbender..."
6,Tangled (2010),"Mandy Moore, Zachary Levi, Donna Murphy"
7,Nappily Ever After (2018),"Sanaa Lathan, Ernie Hudson, Lyriq Bent, Lynn W..."
8,Jurassic World (2015),"Chris Pratt, Bryce Dallas Howard, Vincent DOno..."
9,Gnomeo & Juliet (2011),"James McAvoy, Emily Blunt, Michael Caine, Magg..."


In [253]:
df_new = pd.DataFrame(df_dir["starring"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [254]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [255]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
starring = df_melt_long
starring['edge_type'] = "Starred In"
starring["from_type"] = "Movie"
starring["to_type"] = "Actor"

In [256]:
starring.head()

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),John Huston,Starred In,Movie,Actor
1,Pál Adrienn (2010),Éva Gábor,Starred In,Movie,Actor
2,Foodfight! (2012),Charlie Sheen,Starred In,Movie,Actor
3,The Wicker Tree (2011),Christopher Lee,Starred In,Movie,Actor
4,Blood and Bone (2009),Julian Sands,Starred In,Movie,Actor


Genres

In [257]:
df_dir = df[['title', 'genres']]
df_dir['genres'] = df_dir['genres'].str.replace(r"[\"\'\\[\]]", '')
df_dir = df_dir[df_dir['genres'].notnull()]
df_dir = df_dir.reset_index(drop=True)
df_dir.head(10)

<ipython-input-257-4e8ddb9f4409>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['genres'] = df_dir['genres'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-257-4e8ddb9f4409>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir['genres'] = df_dir['genres'].str.replace(r"[\"\'\\[\]]", '')


,title,genres
0,The Other Side of the Wind (2018),Drama
1,Pál Adrienn (2010),Drama
2,Foodfight! (2012),"Action,Adventure,Animation"
3,Chinango (2009),"Action,Comedy"
4,Wazir (2016),"Action,Crime,Drama"
5,The Wicker Tree (2011),"Drama,Horror"
6,Coraline (2009),"Animation,Drama,Family"
7,On the Road (2012),"Adventure,Drama,Romance"
8,The Future (2011),"Drama,Fantasy,Romance"
9,The Watch (2012),"Comedy,Sci-Fi"


In [258]:
df_new = pd.DataFrame(df_dir["genres"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [259]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [260]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
genres = df_melt_long
genres['edge_type'] = "Genre"
genres["from_type"] = "Movie"
genres["to_type"] = "Genre"

In [261]:
genres.head()

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),Drama,Genre,Movie,Genre
1,Pál Adrienn (2010),Drama,Genre,Movie,Genre
2,Foodfight! (2012),Action,Genre,Movie,Genre
3,Chinango (2009),Action,Genre,Movie,Genre
4,Wazir (2016),Action,Genre,Movie,Genre


Producer

In [262]:
df_dir = df[['title', 'producer']]
df_dir = df_dir[df_dir['producer'].notna()]
df_dir = df_dir[df_dir["producer"].str.contains("\n")]
df_dir['producer'] = df_dir['producer'].str.replace(r"[\"\'\\[\]]", '')
df_dir['producer'] = df_dir['producer'].str.replace(r"\n", ',')
df_dir = df_dir[~df_dir["producer"].str.contains("\.")]
df_dir = df_dir[df_dir['producer'].notnull()]
df_dir = df_dir.reset_index(drop=True)
#df_dir = df_dir[0:285]
#df_dir = df_dir.drop_duplicates(subset=['title'], keep='first')
df_dir.head(10)

<ipython-input-262-a052feee989c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['producer'] = df_dir['producer'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-262-a052feee989c>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['producer'] = df_dir['producer'].str.replace(r"\n", ',')


,title,producer
0,Foodfight! (2012),",George Johnsen,Lawrence Kasanoff,Joshua Wexler,"
1,Daybreakers (2009),",Chris Brown,Sean Furst,Bryan Furst,"
2,The Equalizer (2014),",Dan Lieberstein,Marc Laub,Stuart Cohen,Alan B..."
3,Scary Movie 5 (2013),",David Zucker,Phil Dornfeld,"
4,She's Out of My League (2010),",Jimmy Miller,David Householter,"
5,My Life in Ruins (2009),",Michelle Chydzik Sowa,Nathalie Marciano,"
6,My Soul to Take (2010),",Wes Craven,Anthony Katagas,Iya Labunka,"
7,In the Electric Mist (2009),",Frédéric Bourboulon,Michael Fitzgerald,"
8,Snatchers (2019),",Eric Fisher,Scott Hinckley,Elli Legerski,Paul..."
9,Cleanflix (2009),",Amber Bollinger,Andrew James,Joshua Ligairi,"


In [263]:
df_new = pd.DataFrame(df_dir["producer"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [264]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [265]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long['to'] = df_melt_long['to'].str.replace(r"\,", '')
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
producer = df_melt_long
producer['edge_type'] = "Produced"
producer["from_type"] = "Movie"
producer["to_type"] = "Producer"

<ipython-input-265-5907ec95a247>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_melt_long['to'] = df_melt_long['to'].str.replace(r"\,", '')


In [266]:
producer.head()

,from,to,edge_type,from_type,to_type
141,Upstream Color (2013),Shane Carruth,Produced,Movie,Producer
260,The Man from Earth: Holocene (2017),Emerson Bixby,Produced,Movie,Producer
383,Foodfight! (2012),George Johnsen,Produced,Movie,Producer
384,Daybreakers (2009),Chris Brown,Produced,Movie,Producer
385,The Equalizer (2014),Dan Lieberstein,Produced,Movie,Producer


Writers

In [267]:
df_dir = df[['title', 'writers']]
df_dir['writers'] = df_dir['writers'].str.replace(r"[\"\'\\[\]]", '')
df_dir = df_dir[df_dir['writers'].notnull()]
df_dir = df_dir.reset_index(drop=True)
df_dir.head(10)

<ipython-input-267-6164a24bf5dc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['writers'] = df_dir['writers'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-267-6164a24bf5dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir['writers'] = df_dir['writers'].str.replace(r"[\"\'\\[\]]", '')


,title,writers
0,The Other Side of the Wind (2018),"nm0000080,nm0462648"
1,Pál Adrienn (2010),"nm1030585,nm1921430"
2,Foodfight! (2012),"nm0440415,nm0923312,nm0295165,nm0841854,nm0220297"
3,Chinango (2009),"nm1691085,nm1138779"
4,Wazir (2016),"nm0006765,nm0430785,nm2349060,nm5241801,nm1249..."
5,The Wicker Tree (2011),nm0362736
6,Coraline (2009),"nm0783139,nm0301274"
7,On the Road (2012),"nm0449616,nm1433580"
8,The Future (2011),nm0432380
9,The Watch (2012),"nm2972864,nm0736622,nm1698571"


In [268]:
df_new = pd.DataFrame(df_dir["writers"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [269]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [270]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
writers = df_melt_long
writers['edge_type'] = "Wrote"
writers["from_type"] = "Movie"
writers["to_type"] = "Writer"

In [271]:
writers.head()

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),nm0000080,Wrote,Movie,Writer
1,Pál Adrienn (2010),nm1030585,Wrote,Movie,Writer
2,Foodfight! (2012),nm0440415,Wrote,Movie,Writer
3,Chinango (2009),nm1691085,Wrote,Movie,Writer
4,Wazir (2016),nm0006765,Wrote,Movie,Writer


Characters

In [272]:
df_dir = df[['title', 'characters']]
df_dir['characters'] = df_dir['characters'].str.replace(r"[\"\'\\[\]]", '')
df_dir = df_dir[df_dir['characters'].notnull()]
df_dir = df_dir.reset_index(drop=True)
df_dir.head(10)

<ipython-input-272-a128d6dfd3ff>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dir['characters'] = df_dir['characters'].str.replace(r"[\"\'\\[\]]", '')
<ipython-input-272-a128d6dfd3ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir['characters'] = df_dir['characters'].str.replace(r"[\"\'\\[\]]", '')


,title,characters
0,About Sunny (2011),"Bryan Wizemann, Mike S. Ryan, Lauren Ambrose, ..."
1,The Forest of Love (2019),"Ainaki Mori de Sakebe, Sion Sono, Futoshi Mats..."
2,Mary and Max (2009),"Mary, Max, Adam Elliot, Melanie Coombs, Dale C..."
3,The Lehman Trilogy (2019),"Stefano Massini, Luca Ronconi, Sam Mendes"
4,Noura's Dream (2019),"Noura Rêve, نورا تحلم, Marie Besson, Tatjana K..."
5,Kim Ji-young: Born 1982 (2019),"Kim Ji-young, Cho Nam-Joo, Cho, Kim Ji-youngs,..."
6,My Prince Edward (2019),"Gam1 dou1, Norris Wong, Stephy Tang, Chu Pak-h..."
7,Ashfall (2019),"Ash, Ashfall Fossil Beds State"
8,Poetry (2010),"metre −, Odyssey, Dante, Rumi"
9,Miles Davis: Birth of the Cool (2019),"Miles Davis, Stanley Nelson Jr., Nelson, Davis..."


In [273]:
df_new = pd.DataFrame(df_dir["characters"].apply(splitter).to_list())
df_melt = pd.merge(df_dir, df_new, left_index=True, right_index=True)

In [274]:
cols = df_melt.columns.tolist()
indices = [0]
id_cols = [cols[i] for i in indices]
v_indices = range(2,len(cols))
value_vars = [cols[i] for i in v_indices]

In [275]:
df_melt_long = df_melt.melt(id_vars=id_cols, value_vars=value_vars, var_name="remove", value_name='to', ignore_index=True)
df_melt_long = df_melt_long[df_melt_long ['to'] != ""]
df_melt_long = df_melt_long[df_melt_long ['to'] != None]
df_melt_long = df_melt_long[df_melt_long ['to'].notnull()]
df_melt_long = df_melt_long.drop(['remove'], axis=1)
df_melt_long = df_melt_long.rename(columns={'title': 'from'})
characters = df_melt_long
characters['edge_type'] = "Character In"
characters["from_type"] = "Movie"
characters["to_type"] = "Character"

In [276]:
characters.head()

,from,to,edge_type,from_type,to_type
0,About Sunny (2011),Bryan Wizemann,Character In,Movie,Character
1,The Forest of Love (2019),Ainaki Mori de Sakebe,Character In,Movie,Character
2,Mary and Max (2009),Mary,Character In,Movie,Character
3,The Lehman Trilogy (2019),Stefano Massini,Character In,Movie,Character
4,Noura's Dream (2019),Noura Rêve,Character In,Movie,Character


Directed-Starred

In [277]:
directed_starred = pd.merge(directors, starring, how="left", on="from")
directed_starred = directed_starred[['to_x','to_y']]
directed_starred = directed_starred.rename(columns={'to_x': 'from', 'to_y':'to'})
directed_starred['edge_type'] = "Directed Actor"
directed_starred["from_type"] = "Director"
directed_starred["to_type"] = "Actor"
directed_starred.head()

,from,to,edge_type,from_type,to_type
0,Orson Welles,John Huston,Directed Actor,Director,Actor
1,Orson Welles,Peter Bogdanovich,Directed Actor,Director,Actor
2,Orson Welles,Susan Strasberg,Directed Actor,Director,Actor
3,Orson Welles,Norman Foster,Directed Actor,Director,Actor
4,Orson Welles,Bob Random,Directed Actor,Director,Actor


Directed_Genre

In [278]:
directed_genre = pd.merge(directors, genres, how="left", on="from")
directed_genre = directed_genre[['to_x','to_y']]
directed_genre = directed_genre.rename(columns={'to_x': 'from', 'to_y':'to'})
directed_genre['edge_type'] = "Directed Genre"
directed_genre["from_type"] = "Director"
directed_genre["to_type"] = "Genre"
directed_genre.head()

,from,to,edge_type,from_type,to_type
0,Orson Welles,Drama,Directed Genre,Director,Genre
1,Ágnes Kocsis,Drama,Directed Genre,Director,Genre
2,Lawrence Kasanoff,Action,Directed Genre,Director,Genre
3,Lawrence Kasanoff,Adventure,Directed Genre,Director,Genre
4,Lawrence Kasanoff,Animation,Directed Genre,Director,Genre


Produced-Starred

In [279]:
producer_starred = pd.merge(producer, starring, how="left", on="from")
producer_starred = producer_starred[['to_x','to_y']]
producer_starred = producer_starred.rename(columns={'to_x': 'from', 'to_y':'to'})
producer_starred['edge_type'] = "Produced Movie with Actor"
producer_starred["from_type"] = "Producer"
producer_starred["to_type"] = "Actor"
producer_starred.head()

,from,to,edge_type,from_type,to_type
0,Shane Carruth,Amy Seimetz,Produced Movie with Actor,Producer,Actor
1,Emerson Bixby,David Lee Smith,Produced Movie with Actor,Producer,Actor
2,George Johnsen,Charlie Sheen,Produced Movie with Actor,Producer,Actor
3,George Johnsen,Eva Longoria,Produced Movie with Actor,Producer,Actor
4,George Johnsen,Hilary Duff,Produced Movie with Actor,Producer,Actor


Produced-Genre

In [280]:
producer_genre = pd.merge(producer, genres, how="left", on="from")
producer_genre = producer_genre[['to_x','to_y']]
producer_genre = producer_genre.rename(columns={'to_x': 'from', 'to_y':'to'})
producer_genre['edge_type'] = "Produced Genre"
producer_genre["from_type"] = "Producer"
producer_genre["to_type"] = "Genre"
producer_genre.head()

,from,to,edge_type,from_type,to_type
0,Shane Carruth,Drama,Produced Genre,Producer,Genre
1,Shane Carruth,Sci-Fi,Produced Genre,Producer,Genre
2,Emerson Bixby,Drama,Produced Genre,Producer,Genre
3,Emerson Bixby,Fantasy,Produced Genre,Producer,Genre
4,Emerson Bixby,Sci-Fi,Produced Genre,Producer,Genre


Wrote-Starred

In [281]:
wrote_starred = pd.merge(writers, starring, how="left", on="from")
wrote_starred = wrote_starred[['to_x','to_y']]
wrote_starred = wrote_starred.rename(columns={'to_x': 'from', 'to_y':'to'})
wrote_starred['edge_type'] = "Wrote Movie with Actor"
wrote_starred["from_type"] = "Writer"
wrote_starred["to_type"] = "Actor"
wrote_starred.head()

,from,to,edge_type,from_type,to_type
0,nm0000080,John Huston,Wrote Movie with Actor,Writer,Actor
1,nm0000080,Peter Bogdanovich,Wrote Movie with Actor,Writer,Actor
2,nm0000080,Susan Strasberg,Wrote Movie with Actor,Writer,Actor
3,nm0000080,Norman Foster,Wrote Movie with Actor,Writer,Actor
4,nm0000080,Bob Random,Wrote Movie with Actor,Writer,Actor


Wrote-Genre

In [282]:
wrote_genre = pd.merge(writers, genres, how="left", on="from")
wrote_genre = wrote_genre[['to_x','to_y']]
wrote_genre = wrote_genre.rename(columns={'to_x': 'from', 'to_y':'to'})
wrote_genre['edge_type'] = "Wrote Genre"
wrote_genre["from_type"] = "Writer"
wrote_genre["to_type"] = "Genre"
wrote_genre.head()

,from,to,edge_type,from_type,to_type
0,nm0000080,Drama,Wrote Genre,Writer,Genre
1,nm1030585,Drama,Wrote Genre,Writer,Genre
2,nm0440415,Action,Wrote Genre,Writer,Genre
3,nm0440415,Adventure,Wrote Genre,Writer,Genre
4,nm0440415,Animation,Wrote Genre,Writer,Genre


Starred-Genre

In [283]:
starred_genre = pd.merge(starring, genres, how="left", on="from")
starred_genre = starred_genre[['to_x','to_y']]
starred_genre = starred_genre.rename(columns={'to_x': 'from', 'to_y':'to'})
starred_genre['edge_type'] = "Starred in Genre"
starred_genre["from_type"] = "Actor"
starred_genre["to_type"] = "Genre"
starred_genre.head()

,from,to,edge_type,from_type,to_type
0,John Huston,Drama,Starred in Genre,Actor,Genre
1,Éva Gábor,Drama,Starred in Genre,Actor,Genre
2,Charlie Sheen,Action,Starred in Genre,Actor,Genre
3,Charlie Sheen,Adventure,Starred in Genre,Actor,Genre
4,Charlie Sheen,Animation,Starred in Genre,Actor,Genre


Directed-Produced

In [284]:
directed_produced = pd.merge(directors, producer, how="left", on="from")
directed_produced = directed_produced[['to_x','to_y']]
directed_produced = directed_produced.rename(columns={'to_x': 'from', 'to_y':'to'})
directed_produced['edge_type'] = "Directed Movie with Producer"
directed_produced["from_type"] = "Director"
directed_produced["to_type"] = "Producer"
directed_produced.head()

,from,to,edge_type,from_type,to_type
0,Orson Welles,NaN,Directed Movie with Producer,Director,Producer
1,Ágnes Kocsis,NaN,Directed Movie with Producer,Director,Producer
2,Lawrence Kasanoff,George Johnsen,Directed Movie with Producer,Director,Producer
3,Lawrence Kasanoff,Lawrence Kasanoff,Directed Movie with Producer,Director,Producer
4,Lawrence Kasanoff,Joshua Wexler,Directed Movie with Producer,Director,Producer


Directred-Wrote

In [285]:
directed_wrote = pd.merge(directors, writers, how="left", on="from")
directed_wrote = directed_wrote[['to_x','to_y']]
directed_wrote = directed_wrote.rename(columns={'to_x': 'from', 'to_y':'to'})
directed_wrote['edge_type'] = "Directed Movie with Writer"
directed_wrote["from_type"] = "Director"
directed_wrote["to_type"] = "Writer"
directed_wrote.head()

,from,to,edge_type,from_type,to_type
0,Orson Welles,nm0000080,Directed Movie with Writer,Director,Writer
1,Orson Welles,nm0462648,Directed Movie with Writer,Director,Writer
2,Ágnes Kocsis,nm1030585,Directed Movie with Writer,Director,Writer
3,Ágnes Kocsis,nm1921430,Directed Movie with Writer,Director,Writer
4,Lawrence Kasanoff,nm0440415,Directed Movie with Writer,Director,Writer


In [286]:
full_edge = pd.concat([directors,starring,genres,producer,writers,characters, directed_starred, directed_genre, producer_starred, producer_genre, wrote_starred,wrote_genre,starred_genre,directed_produced,directed_wrote ])
full_edge = full_edge[full_edge['from'].notna()]
full_edge = full_edge[full_edge['to'].notna()]
full_edge.head()

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),Orson Welles,Directed,Movie,Director
1,Pál Adrienn (2010),Ágnes Kocsis,Directed,Movie,Director
2,Foodfight! (2012),Lawrence Kasanoff,Directed,Movie,Director
3,Chinango (2009),Peter Van Lengen,Directed,Movie,Director
4,Wazir (2016),Bejoy Nambiar,Directed,Movie,Director


In [302]:
full_edge[full_edge['from'] == "The Other Side of the Wind (2018)"]

,from,to,edge_type,from_type,to_type
0,The Other Side of the Wind (2018),Orson Welles,Directed,Movie,Director
0,The Other Side of the Wind (2018),John Huston,Starred In,Movie,Actor
4630,The Other Side of the Wind (2018),Peter Bogdanovich,Starred In,Movie,Actor
9260,The Other Side of the Wind (2018),Susan Strasberg,Starred In,Movie,Actor
13890,The Other Side of the Wind (2018),Norman Foster,Starred In,Movie,Actor
18520,The Other Side of the Wind (2018),Bob Random,Starred In,Movie,Actor
23150,The Other Side of the Wind (2018),Lilli Palmer,Starred In,Movie,Actor
27780,The Other Side of the Wind (2018),Edmond OBrien,Starred In,Movie,Actor
32410,The Other Side of the Wind (2018),Mercedes McCambridge,Starred In,Movie,Actor
37040,The Other Side of the Wind (2018),Cameron Mitchell,Starred In,Movie,Actor


In [304]:
nodes_n = full_edge[['from','from_type']].drop_duplicates()
nodes_n = nodes_n.rename(columns={'from': 'node', 'from_type':'node_type'})
nodes_n.shape

(64425, 2)

In [305]:
nodes_t = full_edge[['to','to_type']].drop_duplicates()
nodes_t = nodes_n.rename(columns={'to': 'node', 'to_type':'node_type'})
nodes_t.shape

(64425, 2)

In [306]:
nodes = pd.concat([nodes_n,nodes_t])
nodes= nodes.drop_duplicates()
nodes.head()

,node,node_type
0,The Other Side of the Wind (2018),Movie
1,Pál Adrienn (2010),Movie
2,Foodfight! (2012),Movie
3,Chinango (2009),Movie
4,Wazir (2016),Movie


In [308]:
nodes.reset_index(drop=True, inplace=True)
nodes.reset_index(inplace=True)
nodes = nodes.rename(columns={'index': 'id'})
nodes

,id,node,node_type
0,0,The Other Side of the Wind (2018),Movie
1,1,Pál Adrienn (2010),Movie
2,2,Foodfight! (2012),Movie
3,3,Chinango (2009),Movie
4,4,Wazir (2016),Movie
...,...,...,...
64420,64420,Michael Cuccione,Actor
64421,64421,Álvaro Begines,Director
64422,64422,Juan Manuel DEmilio,Director
64423,64423,Johnny Barbuzano,Director


In [309]:
nodes['id'] = nodes['id']+1

In [310]:
nodes['id'] = nodes['id'].astype('str')

Merge node ids with edge list

In [311]:
full_edge_S = pd.merge(full_edge, nodes, how = "left", left_on = ["from"], right_on=["node"])

In [312]:
full_edge_S = pd.merge(full_edge_S, nodes, how = "left", left_on = ["to"], right_on=["node"])

In [314]:
full_edge_S.reset_index(drop=True, inplace=True)
full_edge_S.reset_index(inplace=True)
full_edge_S = full_edge_S.rename(columns={'index': 'id'})
full_edge_S

,id,from,to,edge_type,from_type,to_type,id_x,node_x,node_type_x,id_y,node_y,node_type_y
0,0,The Other Side of the Wind (2018),Orson Welles,Directed,Movie,Director,1,The Other Side of the Wind (2018),Movie,16839,Orson Welles,Director
1,1,Pál Adrienn (2010),Ágnes Kocsis,Directed,Movie,Director,2,Pál Adrienn (2010),Movie,16840,Ágnes Kocsis,Director
2,2,Foodfight! (2012),Lawrence Kasanoff,Directed,Movie,Director,3,Foodfight! (2012),Movie,16841,Lawrence Kasanoff,Director
3,3,Foodfight! (2012),Lawrence Kasanoff,Directed,Movie,Director,3,Foodfight! (2012),Movie,29228,Lawrence Kasanoff,Producer
4,4,Chinango (2009),Peter Van Lengen,Directed,Movie,Director,4,Chinango (2009),Movie,20839,Peter Van Lengen,Director
...,...,...,...,...,...,...,...,...,...,...,...,...
377813,377813,Álex de la Iglesia,nm0620378,Directed Movie with Writer,Director,Writer,61805,Álex de la Iglesia,Actor,52338,nm0620378,Writer
377814,377814,Álex de la Iglesia,nm0861610,Directed Movie with Writer,Director,Writer,28881,Álex de la Iglesia,Director,38639,nm0861610,Writer
377815,377815,Álex de la Iglesia,nm0861610,Directed Movie with Writer,Director,Writer,61805,Álex de la Iglesia,Actor,38639,nm0861610,Writer
377816,377816,Álex de la Iglesia,nm0407067,Directed Movie with Writer,Director,Writer,28881,Álex de la Iglesia,Director,31005,nm0407067,Writer


In [315]:
full_edge_S = full_edge_S[['id','id_x','id_y', 'edge_type']]

In [316]:
full_edge_S = full_edge_S.rename(columns={'id_x': 'from', 'id_y':'to'})

In [318]:
full_edge_S['id'] = full_edge_S['id'] + 1
full_edge_S['id'] = "e" + full_edge_S['id'].astype('str')
full_edge_S['from'] = full_edge_S['from'].astype("str")
full_edge_S['to'] = full_edge_S['to'].astype("str")

In [320]:
full_edge_S.head()

,id,from,to,edge_type
0,e1,1,16839,Directed
1,e2,2,16840,Directed
2,e3,3,16841,Directed
3,e4,3,29228,Directed
4,e5,4,20839,Directed


# Add Edges and Nodes to Neptune Graph

In [321]:
# Necessary to avoid Cannot run the event loop while another loop is running error
import nest_asyncio
nest_asyncio.apply()

In [353]:
graph = Graph()

remoteConn = DriverRemoteConnection('ws://mgt-prd-infr-neptune-alb-122161610.us-east-1.elb.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)

Reset from previous

In [334]:
g.V().drop().iterate()

[['V'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [335]:
g.E().drop().iterate()

[['E'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [336]:
g.V().count().next()

0

In [337]:
g.E().count().next()

0

Add Nodes

In [338]:
len(nodes)

64425

In [339]:
id = T.id
single = Cardinality.single

In [342]:
ct = 0
for i in range(0,len(nodes)):
    g.addV(nodes['node_type'][i]).property(id, nodes['id'][i]). \
    property('node', nodes['node'][i]). \
    next()
    
    ct = ct + 1
    if ct % 15000 == 0:
        print(ct)
        
        #Reconnect
        graph = Graph()
        remoteConn = DriverRemoteConnection('ws://mgt-prd-infr-neptune-alb-122161610.us-east-1.elb.amazonaws.com:8182/gremlin','g')
        g = graph.traversal().withRemote(remoteConn)
    else:
        pass
        

15000
30000
45000
60000


Add Edges

In [348]:
len(full_edge_S)

377818

In [351]:
ct = 0
for i in range(0,len(full_edge_S)):
    g.V().has(id, full_edge_S['to'][i]).\
    as_('v').V().has(id,full_edge_S['from'][i]).coalesce(
    __.inE(full_edge_S['edge_type'][i]).where(__.outV().as_('v')), 
    __.addE(full_edge_S['edge_type'][i]).property(id, full_edge_S['id'][i]).from_('v')).iterate()
    
    ct = ct + 1
    if ct % 15000 == 0:
        print(ct)
        
        #Reconnect
        graph = Graph()
        remoteConn = DriverRemoteConnection('ws://mgt-prd-infr-neptune-alb-122161610.us-east-1.elb.amazonaws.com:8182/gremlin','g')
        g = graph.traversal().withRemote(remoteConn)
    else:
        pass

15000
30000
45000
60000
75000
90000
105000
120000
135000
150000
165000
180000
195000
210000
225000
240000
255000
270000
285000
300000
315000
330000
345000
360000
375000


Check on process success

In [354]:
g.V().count().next()

64425

In [355]:
g.V().limit(10).elementMap().toList()

[{<T.id: 1>: '1',
  <T.label: 4>: 'Movie',
  'node': 'The Other Side of the Wind (2018)'},
 {<T.id: 1>: '2', <T.label: 4>: 'Movie', 'node': 'Pál Adrienn (2010)'},
 {<T.id: 1>: '3', <T.label: 4>: 'Movie', 'node': 'Foodfight! (2012)'},
 {<T.id: 1>: '4', <T.label: 4>: 'Movie', 'node': 'Chinango (2009)'},
 {<T.id: 1>: '5', <T.label: 4>: 'Movie', 'node': 'Wazir (2016)'},
 {<T.id: 1>: '6', <T.label: 4>: 'Movie', 'node': 'The Wicker Tree (2011)'},
 {<T.id: 1>: '7', <T.label: 4>: 'Movie', 'node': 'Coraline (2009)'},
 {<T.id: 1>: '8', <T.label: 4>: 'Movie', 'node': 'On the Road (2012)'},
 {<T.id: 1>: '9', <T.label: 4>: 'Movie', 'node': 'The Future (2011)'},
 {<T.id: 1>: '10', <T.label: 4>: 'Movie', 'node': 'The Watch (2012)'}]

In [356]:
g.E().count().next()

175640

In [357]:
g.E().limit(10).elementMap().toList()

[{<T.id: 1>: 'e14210',
  <T.label: 4>: 'Directed',
  <Direction.IN: 2>: {<T.id: 1>: '13628', <T.label: 4>: 'Movie'},
  <Direction.OUT: 3>: {<T.id: 1>: '27529', <T.label: 4>: 'Director'}},
 {<T.id: 1>: 'e14274',
  <T.label: 4>: 'Directed',
  <Direction.IN: 2>: {<T.id: 1>: '13687', <T.label: 4>: 'Movie'},
  <Direction.OUT: 3>: {<T.id: 1>: '27555', <T.label: 4>: 'Director'}},
 {<T.id: 1>: 'e14338',
  <T.label: 4>: 'Directed',
  <Direction.IN: 2>: {<T.id: 1>: '13748', <T.label: 4>: 'Movie'},
  <Direction.OUT: 3>: {<T.id: 1>: '27244', <T.label: 4>: 'Director'}},
 {<T.id: 1>: 'e14402',
  <T.label: 4>: 'Directed',
  <Direction.IN: 2>: {<T.id: 1>: '13810', <T.label: 4>: 'Movie'},
  <Direction.OUT: 3>: {<T.id: 1>: '26764', <T.label: 4>: 'Director'}},
 {<T.id: 1>: 'e14466',
  <T.label: 4>: 'Directed',
  <Direction.IN: 2>: {<T.id: 1>: '13872', <T.label: 4>: 'Movie'},
  <Direction.OUT: 3>: {<T.id: 1>: '27625', <T.label: 4>: 'Director'}},
 {<T.id: 1>: 'e14530',
  <T.label: 4>: 'Directed',
  <Direct